In [1]:
import gym
import copy
import numpy as np

## Policy Iteration
As defined in the book of Barto and Sutton. 

In [17]:
def policy_evaluation(env, policy, discount_factor=1.0, 
                      theta=1e-3):
    V = np.zeros(env.nS)
    eval_sweeps = 0
    
    while True:
        delta = 0

        for s in range(env.nS):
            v = copy.deepcopy(V[s])
            V_s = 0
            for a, action_prob in enumerate(policy[s]):
                for prob, next_state, reward, _ in env.P[s][a]:
                    V_s += action_prob * prob * \
                        (reward + discount_factor * V[next_state])
            V[s] = V_s

            delta = max(delta, np.abs(v-V[s]))
        eval_sweeps += 1
        print("policy evaluation sweeps:", eval_sweeps)
        if delta < theta:
            break
    return np.array(V)


def policy_improvement(env, policy, V, discount_factor):
    def next_actions(env, s, V):
        actions = np.zeros(env.nA)
        for action in range(env.nA):
            for prob, next_state, reward, _ in env.P[s][action]:
                actions[action] += prob * (reward + discount_factor \
                                      * V[next_state])
        return actions
    
    policy_stable = True
    for s in range(env.nS):
        old_action = np.argmax(policy[s])
        action_values = next_actions(env, s, V)
        best_action = np.argmax(action_values)
        # Keep the actions sparse
        policy[s] = np.eye(env.nA)[best_action]
        
        if best_action != old_action:
            policy_stable = False
    return V, policy, policy_stable


def policy_iteration(env, discount_factor=1.0):
    # initial equiprobable policy
    policy = np.ones([env.nS, env.nA]) / env.nA
    iteration = 0
    policy_stable = False
    
    # Value function initiated in policy evaluation
    while not policy_stable:
        V = policy_evaluation(env, policy, discount_factor)
        V, policy, policy_stable = policy_improvement(env, policy, V, discount_factor)
        iteration += 1
        
        print("Policy:", policy)
        print("Value function:", V)
        print("iteration:", iteration)
    return V, policy

env = gym.make('FrozenLake-v0', is_slippery=False)
V, policy = policy_iteration(env, discount_factor=0.99)

policy evaluation sweeps: 1
policy evaluation sweeps: 2
policy evaluation sweeps: 3
policy evaluation sweeps: 4
policy evaluation sweeps: 5
policy evaluation sweeps: 6
policy evaluation sweeps: 7
policy evaluation sweeps: 8
policy evaluation sweeps: 9
policy evaluation sweeps: 10
policy evaluation sweeps: 11
policy evaluation sweeps: 12
policy evaluation sweeps: 13
Policy: [[0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]]
Value function: [0.00962919 0.0088378  0.01811819 0.0086038  0.01337352 0.
 0.03847913 0.         0.03181995 0.08390147 0.13750157 0.
 0.         0.17000914 0.43332128 0.        ]
iteration: 1
policy evaluation sweeps: 1
policy evaluation sweeps: 2
policy evaluation sweeps: 3
policy evaluation sweeps: 4
policy evaluation sweeps: 5
policy evaluation sweeps: 6
policy evaluation swee

## Value Iteration
By modifying Policy Iteration slightly, the adjustment of taking the max action of the next state to characterize the value function we obtain faster convergence. 

In [15]:
def value_iteration(env, discount_factor=1.0, 
                      theta=1e-3):
    def next_actions(s, V):
        actions = np.zeros(env.nA)
        for action in range(env.nA):
            for prob, next_state, reward, _ in env.P[s][action]:
                actions[action] += prob * (reward + discount_factor \
                                      * V[next_state])
        return actions
    
    # Define an equiprobable policy over all states
    policy = np.ones([env.nS, env.nA]) / env.nA
    iteration = 0
    V = np.zeros(env.nS)
    
    while True:
        delta = 0

        # We can update our policy in the same loop
        for s in range(env.nS):
            v = copy.deepcopy(V[s])
            v_values = next_actions(s, V)
            best_action = np.max(v_values)
            V[s] = best_action
            
            # Update the policy with the best action
            policy[s] = np.eye(env.nA)[np.argmax(v_values)]
        
            delta = max(delta, np.abs(v-V[s]))
        if delta < theta:
            break
        # Print out iterations to compare with Policy
        iteration += 1
        print("Policy:", policy)
        print("Value function:", V)
        print("iteration:", iteration)
    
    return np.array(V), policy


env = gym.make('FrozenLake-v0', is_slippery=False)
V, policy = value_iteration(env, discount_factor=0.99)

policy: [[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]]
Value function: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
Iteration: 1
policy: [[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]]
Value function: [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.99 0.   0.   0.99
 1.   0.  ]
Iteration: 2
policy: [[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]]
Value function: [0.     0.     0.     0.     0.     0.     0.9

Comparing the iterations required we observe that value iteration even in this trivial example requires less iterations, compared to policy iteration.

Furthermore, both policies are equal and yield the same behavior. 

In [21]:
def act_in_environment(env, policy, render=False):
    state = env.reset()
    episode_rew = 0

    for step in range(env._max_episode_steps):
        action = np.argmax(policy[state])
        state, reward, done, _ = env.step(action)

        if render:
            env.render()

        episode_rew += reward

        if done:
            print(
                f"Finished the episode with {episode_rew} \
                    reward in {step} steps")
            break
    return step, episode_rew


env = gym.make('FrozenLake-v0', is_slippery=False)
V, policy = policy_iteration(env, discount_factor=0.99)

step, reward = act_in_environment(env, policy, render=True)
f"Solved in {step} steps and acquired {reward} reward."

policy evaluation sweeps: 1
policy evaluation sweeps: 2
policy evaluation sweeps: 3
policy evaluation sweeps: 4
policy evaluation sweeps: 5
policy evaluation sweeps: 6
policy evaluation sweeps: 7
policy evaluation sweeps: 8
policy evaluation sweeps: 9
policy evaluation sweeps: 10
policy evaluation sweeps: 11
policy evaluation sweeps: 12
policy evaluation sweeps: 13
Policy: [[0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]]
Value function: [0.00962919 0.0088378  0.01811819 0.0086038  0.01337352 0.
 0.03847913 0.         0.03181995 0.08390147 0.13750157 0.
 0.         0.17000914 0.43332128 0.        ]
iteration: 1
policy evaluation sweeps: 1
policy evaluation sweeps: 2
policy evaluation sweeps: 3
policy evaluation sweeps: 4
policy evaluation sweeps: 5
policy evaluation sweeps: 6
policy evaluation swee

'In 5 steps and acquired 1.0 reward.'

In [22]:
V, policy = value_iteration(env, discount_factor=0.99)
step, reward = act_in_environment(env, policy, render=True)
f"Solved in {step} steps and acquired {reward} reward."

policy: [[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]]
Value function: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
Iteration: 1
policy: [[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]]
Value function: [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.99 0.   0.   0.99
 1.   0.  ]
Iteration: 2
policy: [[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]]
Value function: [0.     0.     0.     0.     0.     0.     0.9

'Solved in 5 steps and acquired 1.0 reward.'